# Environment setting
環境設定

In [1]:
import os
import requests
from pathlib import Path


# determine branch, default is main
branch = 'main'

# Check if running in Google Colab
is_colab = 'COLAB_GPU' in os.environ

if is_colab:
    # Download the utils.py file from GitHub
    utils_url = f"https://raw.githubusercontent.com/nics-tw/petsard/{branch}/demo/utils.py"
    response = requests.get(utils_url)

    if response.status_code == 200:
        # Save the utils.py file
        with open('utils.py', 'w') as f:
            f.write(response.text)

        # Create an empty __init__.py
        Path('__init__.py').touch()
    else:
        raise RuntimeError(f"Failed to download utils.py. Status code: {response.status_code}")

In [2]:
# Now import and run the setup
from utils import (
    get_yaml_path,
    setup_environment,
)


setup_environment(
    is_colab,
    branch,
    benchmark_data=[
        'adult-income',
    ]
)

Looking in links: /var/folders/lb/lwjvbr314wj7bt1k63plw4bw0000gn/T/tmpy022ouso
Obtaining file:///Users/justyn.chen/Dropbox/310_Career_%E5%B7%A5%E4%BD%9C/20231016_NICS_%E8%B3%87%E5%AE%89%E9%99%A2/41_PETsARD/petsard
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Checking if build backend supports build_editable: started
  Checking if build backend supports build_editable: finished with status 'done'
  Getting requirements to build editable: started
  Getting requirements to build editable: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing editable metadata (pyproject.toml): started
  Preparing editable metadata (pyproject.toml): finished with status 'done'
  Building editable for petsard (pyproject.toml): started
  Building editable for petsard (pyproject.toml): finished with status 'done'
  Created wheel for petsard: filename=petsard-1.

In [3]:
from petsard import Executor

# Best practices: High-Cardinality and Multi-Table Data
最佳實踐：高基數與多表格資料的合成

In [4]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

## Enterprise Basic Information
企業基本資料

In [5]:
# 企業基本資料 (Enterprise Basic Information)
def generate_company_data(n_companies=1000):
    # 產業分類（高基數示範）(Industry Categories - High Cardinality Example)
    industry_types = {
        "製造業": ["電子零組件", "金屬加工", "紡織", "食品", "塑膠製品"],
        "服務業": ["餐飲", "物流", "教育", "休閒娛樂", "專業諮詢"],
        "批發零售": ["電子商務", "進出口貿易", "零售", "汽機車零件", "民生用品"],
        "營建工程": ["土木工程", "建築工程", "室內裝修", "機電工程", "環保工程"]
    }

    # 縣市與區域 (Cities and Districts)
    locations = {
        "臺北市": ["大安區", "信義區", "內湖區"],
        "新北市": ["板橋區", "三重區", "新莊區"],
        "桃園市": ["中壢區", "桃園區", "龜山區"]
    }

    companies = []
    for i in range(n_companies):
        industry = np.random.choice(list(industry_types.keys()))
        sub_industry = np.random.choice(industry_types[industry])
        city = np.random.choice(list(locations.keys()))
        district = np.random.choice(locations[city])

        # 生成隨機成立日期（2010-2020年間）
        # Generate random establishment date (between 2010-2020)
        established_date = datetime(2010, 1, 1) + timedelta(
            days=np.random.randint(0, 365*10)
        )

        company = {
            "company_id": f"C{str(i+1).zfill(6)}",
            "industry": industry,
            "sub_industry": sub_industry,
            "city": city,
            "district": district,
            "established_date": established_date,
            "capital": np.random.randint(1000, 50000) * 1000  # 資本額（以千為單位）(Capital in thousands)
        }
        companies.append(company)

    return pd.DataFrame(companies)


companies = generate_company_data(1000)

print("\n企業基本資料前5筆 (Top 5 records of Enterprise)：\n")
print(companies.head())


企業基本資料前5筆 (Top 5 records of Enterprise)：

  company_id industry sub_industry city district established_date   capital
0    C000001     營建工程         建築工程  新北市      板橋區       2014-04-23  34900000
1    C000002      製造業         塑膠製品  臺北市      信義區       2015-06-13  47283000
2    C000003     營建工程         土木工程  新北市      新莊區       2012-09-24  21751000
3    C000004      製造業           紡織  新北市      板橋區       2014-02-20   2002000
4    C000005      製造業           紡織  臺北市      信義區       2016-07-19   8298000


## Financing Application Records
融資申請紀錄

In [6]:
# 融資申請紀錄 (Financing Application Records)
def generate_application_data(companies_df, years_of_data=10):
   # 最多十年 (Maximum 10 years)
   applications = []
   # 貸款類型 (Loan Types)
   loan_types = [
       "營運週轉金", "購置機器設備", "廠房擴充",
       "創新研發", "數位轉型", "疫後紓困"
   ]

   for _, company in companies_df.iterrows():
       # 每家公司可能有0-5次申請 (Each company may have 0-5 applications)
       n_applications = np.random.randint(0, 6)
       for j in range(n_applications):
           # 以季為單位計算申請時間 (Calculate application time by quarters)
           # 最多40季 (Maximum 40 quarters)
           quarters = np.random.randint(1, years_of_data * 4)
           apply_date = company['established_date'] + timedelta(
               days=quarters * 90
           )

           # 處理天數 1-60天 (Processing period: 1-60 days)
           process_days = np.random.randint(1, 61)
           approval_date = apply_date + timedelta(days=process_days)

           # 申請狀態 (Application Status)
           status = np.random.choice(
               ["approved", "rejected", "withdrawn"],
               p=[0.75, 0.15, 0.10]
           )

           # 申請金額（以千為單位並轉換為實際金額）
           # (Requested amount in thousands converted to actual amount)
           amount_requested = np.random.randint(500, 20000) * 1000

           application = {
               "application_id": f"A{str(len(applications)+1).zfill(8)}",
               "company_id": company['company_id'],
               "loan_type": np.random.choice(loan_types),
               "apply_date": apply_date,
               "approval_date": approval_date if status == "approved" else None,
               "status": status,
               "amount_requested": amount_requested,
               "amount_approved": None
           }

           if status == "approved":
                # 核准金額為申請金額的60-100%，以千為單位四捨六入五隨機
                # (Approved amount is 60-100% of requested amount,
                # rounded to thousands with random rounding at 5)
                raw_amount = amount_requested * np.random.uniform(0.6, 1.0)
                application["amount_approved"] = int(round(raw_amount / 1000)) * 1000

           applications.append(application)

   return pd.DataFrame(applications)


applications = generate_application_data(companies)

print("\n融資申請紀錄前5筆 (Top 5 records of Financing Application)：\n")
print(applications.head())


融資申請紀錄前5筆 (Top 5 records of Financing Application)：

  application_id company_id loan_type apply_date approval_date    status  \
0      A00000001    C000001     營運週轉金 2017-04-07    2017-05-24  approved   
1      A00000002    C000001      廠房擴充 2015-07-17    2015-08-04  approved   
2      A00000003    C000001      疫後紓困 2016-10-09    2016-10-29  approved   
3      A00000004    C000001      廠房擴充 2020-09-18           NaT  rejected   
4      A00000005    C000001      創新研發 2023-03-07    2023-04-29  approved   

   amount_requested  amount_approved  
0          19302000       14177000.0  
1          13126000       12902000.0  
2          10180000        8305000.0  
3           5510000              NaN  
4           9475000        7152000.0  


## Financial Tracking Records
財務追蹤紀錄

In [7]:
# 財務追蹤紀錄 (Financial Tracking Records)
def generate_financial_tracking(companies_df, applications_df):
    tracking_records = []

    # 篩選核准的申請案 (Filter approved applications)
    approved_applications = applications_df[
        applications_df['status'] == "approved"
    ]

    for _, application in approved_applications.iterrows():
        # 每季追蹤一次，追蹤4-12次 (Quarterly tracking, 4-12 times)
        n_tracking = np.random.randint(4, 13)

        for i in range(n_tracking):
            track_date = application['approval_date'] + timedelta(
                days=90*i
            )

            # 營收與獲利（以千為單位）(Revenue and profit in thousands)
            base_revenue = np.random.randint(1000, 50000) * 1000
            growth_factor = np.random.uniform(0.8, 1.2)
            revenue = int(base_revenue * growth_factor)
            profit = int(revenue * np.random.uniform(-0.1, 0.2))  # 可能虧損 (Possible loss)

            # 根據獲利率決定風險等級 (Determine risk level based on profit ratio)
            profit_ratio = profit / revenue

            # 獲利率分級標準 (Profit ratio thresholds)
            if profit_ratio >= 0.05:
              risk_level = "normal"            # 獲利率 >= 5%
            elif profit_ratio >= 0.02:
              risk_level = "attention"         # 獲利率 2-5%
            elif profit_ratio >= 0:
              risk_level = "warning"           # 獲利率 0-2%
            elif profit_ratio >= -0.03:
              risk_level = "high_risk"         # 獲利率 -3-0%
            elif profit_ratio >= -0.05:
              risk_level = "critical"          # 獲利率 -5--3%
            else:
              risk_level = "severe"            # 獲利率 < -5%

            tracking = {
                "track_id": f"T{str(len(tracking_records)+1).zfill(8)}",
                "application_id": application['application_id'],
                "company_id": application['company_id'],
                "tracking_date": track_date,
                "revenue": revenue,
                "profit": profit,
                "profit_ratio": profit_ratio,
                "risk_level": risk_level
            }

            tracking_records.append(tracking)

    return pd.DataFrame(tracking_records)


tracking = generate_financial_tracking(companies, applications)

print("\n財務追蹤紀錄前5筆 (Top 5 records of Financial Tracking)：\n")
print(tracking.head())


財務追蹤紀錄前5筆 (Top 5 records of Financial Tracking)：

    track_id application_id company_id tracking_date   revenue   profit  \
0  T00000001      A00000001    C000001    2017-05-24  19994506   112186   
1  T00000002      A00000001    C000001    2017-08-22  38804440  1778986   
2  T00000003      A00000001    C000001    2017-11-20  34722275  1226887   
3  T00000004      A00000001    C000001    2018-02-18  31719824 -1317821   
4  T00000005      A00000001    C000001    2018-05-19  40054290  6948779   

   profit_ratio risk_level  
0      0.005611    warning  
1      0.045845  attention  
2      0.035334  attention  
3     -0.041546   critical  
4      0.173484     normal  


# Database Denormalization Preprocessing
資料庫反正規化前處理

In [8]:
def summarize_tracking_data(tracking_df):
    """
    彙整每個申請案的財務追蹤狀況 (Summarize financial tracking status for each application)
    """

    # 將風險等級轉換為數值以計算平均 (Convert risk levels to numeric for calculation)
    risk_level_map = {
        'normal': 1,
        'attention': 2,
        'warning': 3,
        'high_risk': 4,
        'critical': 5,
        'severe': 6
    }

    # 檢查資料欄位是否存在 (Check if columns exist)
    expected_columns = ['risk_level', 'profit_ratio', 'tracking_date', 'revenue']
    for col in expected_columns:
        if col not in tracking_df.columns:
            raise ValueError(f"欄位 '{col}' 不存在於追蹤資料中 (Column '{col}' not found in tracking data)")

    tracking_df['risk_level_num'] = tracking_df['risk_level'].map(risk_level_map)

    summary = tracking_df.groupby('application_id').agg({
        'risk_level_num': [
            ('avg_risk_3y', lambda x: x.iloc[-12:].mean() if len(x) >= 12 else x.mean()),
            ('last_risk', 'last'),
            ('second_last_risk', lambda x: x.iloc[-2] if len(x) >= 2 else None)
        ],
        'profit_ratio': [
            ('avg_profit_ratio', 'mean'),
            ('min_profit_ratio', 'min'),
            ('profit_ratio_std', 'std'),
            ('negative_profit_count', lambda x: (x < 0).sum())
        ],
        'tracking_date': [
            ('tracking_months', lambda x: (x.max() - x.min()).days / 30),
            ('last_tracking_date', 'max')
        ],
        'revenue': [
            ('avg_revenue', 'mean'),
            ('revenue_growth', lambda x: (x.iloc[-1] / x.iloc[0] - 1) if len(x) >= 2 else 0)
        ]
    }).reset_index()

    # 將多層次欄位名稱合併 (Flatten multi-level column names)
    summary.columns = ['_'.join(col).strip('_') for col in summary.columns.values]

    # 將風險等級轉回文字 (Convert risk levels back to text)
    risk_level_reverse_map = {v: k for k, v in risk_level_map.items()}
    for col in ['risk_level_num_last_risk', 'risk_level_num_second_last_risk']:
        summary[col.replace('num_', '')] = summary[col].map(risk_level_reverse_map)

    # 移除中間計算用的欄位 (Remove intermediate calculation columns)
    summary = summary.drop(columns=[col for col in summary.columns if 'num_' in col])

    return summary


def integrate_all_data(companies_df, applications_df, tracking_df):
    """
    整合所有資料表成為申請層級的寬表
    (Integrate all tables into an application-level wide table)
    """
    # 先彙整追蹤資料 (First summarize tracking data)
    tracking_summary = summarize_tracking_data(tracking_df)

    # 合併企業資料與申請資料 (Merge company and application data)
    merged_df = pd.merge(
        applications_df,
        companies_df,
        on='company_id',
        how='left'
    )

    # 加入彙整後的追蹤資料 (Add summarized tracking data)
    final_df = pd.merge(
        merged_df,
        tracking_summary,
        on='application_id',
        how='left'
    )

    return final_df


# 合併資料與彙整追蹤紀錄 (Merge data and summarize tracking records)
# 注意: 這段前處理需在 PETsARD 合成流程之前執行
# (Note: This preprocessing must be executed before PETsARD synthesis process)
denorm_data = integrate_all_data(
   companies,
   applications,
   tracking,
)

# 檢視結果 (Review results)
print("資料形狀 (Data shape):", denorm_data.shape)
print("\n前五筆資料 (First five rows):")
print(denorm_data.head())
print("\n欄位清單 (Column list):")
print(denorm_data.columns.tolist())

# 將反正規化結果儲存 (Save denormalized result)
denorm_data.to_csv('best-practices-high-cardinality-multi-table.csv', index=False)

# 接著可以使用 PETsARD 進行資料合成 (Now you can proceed with PETsARD data synthesis)

資料形狀 (Data shape): (2515, 24)

前五筆資料 (First five rows):
  application_id company_id loan_type apply_date approval_date    status  \
0      A00000001    C000001     營運週轉金 2017-04-07    2017-05-24  approved   
1      A00000002    C000001      廠房擴充 2015-07-17    2015-08-04  approved   
2      A00000003    C000001      疫後紓困 2016-10-09    2016-10-29  approved   
3      A00000004    C000001      廠房擴充 2020-09-18           NaT  rejected   
4      A00000005    C000001      創新研發 2023-03-07    2023-04-29  approved   

   amount_requested  amount_approved industry sub_industry  ...  \
0          19302000       14177000.0     營建工程         建築工程  ...   
1          13126000       12902000.0     營建工程         建築工程  ...   
2          10180000        8305000.0     營建工程         建築工程  ...   
3           5510000              NaN     營建工程         建築工程  ...   
4           9475000        7152000.0     營建工程         建築工程  ...   

  profit_ratio_avg_profit_ratio profit_ratio_min_profit_ratio  \
0                  

# YAML Configuration for PETsARD
PETsARD 的 YAML 設定

In [9]:
yaml_file_case: str = 'best-practices-high-cardinality-multi-table.yaml'

yaml_path_case: str = get_yaml_path(
    is_colab=is_colab,
    yaml_file=yaml_file_case,
    branch=branch,
)

Configuration content:
---
Loader:
  data:
    filepath: 'best-practices-high-cardinality-multi-table.csv'
Preprocessor:
  demo:
    method: 'default'
    config:
      scaler:
        'established_date':
          method: 'scaler_timeanchor'
          reference:
            - 'apply_date'
            - 'approval_date'
            - 'tracking_date_last_tracking_date'
          unit: 'D'
Synthesizer:
  demo:
    method: 'default'
Postprocessor:
  demo:
    method: 'default'
Constrainer:
  demo:
    nan_groups:
      company_id: delete
      # 若公司編號遺失，刪除整筆資料
      # (If company ID is missing, delete entire record)

      industry:
        erase: 'sub_industry'
      # 若主要產業別遺失，擦除子產業別
      # (If main industry is missing, erase sub-industry)

      approval_date:
        erase: ['risk_level_last_risk', 'risk_level_second_last_risk']
      # 若核准日期遺失，清除風險評級相關欄位
      # (If approval date is missing, clear risk rating related fields)

    field_constraints:
      - "established_date <= apply_

### Execution and Result
執行與結果

In [10]:
# Initialize and run executor
exec_case = Executor(config=yaml_path_case)
exec_case.run()

Synthesizer (SDV): Fitting GaussianCopula.


/Users/justyn.chen/Dropbox/310_Career_工作/20231016_NICS_資安院/41_PETsARD/petsard/.venv/lib/python3.10/site-packages/sdv/single_table/base.py:119: FutureWarning: The 'SingleTableMetadata' is deprecated. Please use the new 'Metadata' class for synthesizers.
  warnings.warn(DEPRECATION_MSG, FutureWarning)


Synthesizer (SDV): Fitting GaussianCopula spent 0.6127 sec.
Synthesizer (SDV): Sampling GaussianCopula # 2515 rows (same as Loader data) in 0.1747 sec.


INFO:root:application_id changes data dtype from category[object] to category[object] for metadata alignment.
INFO:root:company_id changes data dtype from category[object] to category[object] for metadata alignment.
INFO:root:loan_type changes data dtype from category[object] to category[object] for metadata alignment.
INFO:root:apply_date changes data dtype from float64 to datetime64[s] for metadata alignment.
INFO:root:approval_date changes data dtype from float64 to datetime64[s] for metadata alignment.
INFO:root:status changes data dtype from category[object] to category[object] for metadata alignment.
INFO:root:amount_requested changes data dtype from float64 to int32 for metadata alignment.
INFO:root:industry changes data dtype from category[object] to category[object] for metadata alignment.
INFO:root:sub_industry changes data dtype from category[object] to category[object] for metadata alignment.
INFO:root:city changes data dtype from category[object] to category[object] for me

Synthesizer (SDV): Sampling GaussianCopula # 25150 rows (same as manual input) in 0.619 sec.


INFO:SingleTableSynthesizer:{'EVENT': 'Sample', 'TIMESTAMP': datetime.datetime(2025, 2, 11, 16, 28, 31, 479130), 'SYNTHESIZER CLASS NAME': 'GaussianCopulaSynthesizer', 'SYNTHESIZER ID': 'GaussianCopulaSynthesizer_1.17.4_fcc95b58987c48d1b47453c5e03b646b', 'TOTAL NUMBER OF TABLES': 1, 'TOTAL NUMBER OF ROWS': 25150, 'TOTAL NUMBER OF COLUMNS': 24}
INFO:PETsARD.Processor:MediatorEncoder is created.
INFO:PETsARD.Processor:MediatorScaler is created.
INFO:PETsARD.Processor:MediatorScaler transformation done.
INFO:PETsARD.Processor:str inverse transformation done.
INFO:PETsARD.Processor:MediatorEncoder transformation done.
INFO:PETsARD.Processor:str inverse transformation done.
INFO:PETsARD.Processor:str inverse transformation done.
INFO:root:application_id changes data dtype from category[object] to category[object] for metadata alignment.
INFO:root:company_id changes data dtype from category[object] to category[object] for metadata alignment.
INFO:root:loan_type changes data dtype from catego

Synthesizer (SDV): Sampling GaussianCopula # 25150 rows (same as manual input) in 0.5923 sec.
Now is petsard_Loader[data]_Preprocessor[demo]_Synthesizer[demo]_Postprocessor[demo]_Constrainer[demo] save to csv...


In [11]:
exec_case.get_result()[
    'Loader[data]_Preprocessor[demo]_Synthesizer[demo]_Postprocessor[demo]_Constrainer[demo]_Reporter[output]'
]['Loader[data]_Preprocessor[demo]_Synthesizer[demo]_Postprocessor[demo]_Constrainer[demo]']

,application_id,company_id,loan_type,apply_date,approval_date,status,amount_requested,amount_approved,industry,sub_industry,...,profit_ratio_avg_profit_ratio,profit_ratio_min_profit_ratio,profit_ratio_profit_ratio_std,profit_ratio_negative_profit_count,tracking_date_tracking_months,tracking_date_last_tracking_date,revenue_avg_revenue,revenue_revenue_growth,risk_level_last_risk,risk_level_second_last_risk
0,A00001610,C000034,購置機器設備,2017-10-22 11:31:54,2017-12-29 17:36:00,approved,16343938,13241359.00,製造業,電子零組件,...,0.014727,-0.094265,0.012475,5.468361,28.486275,2020-05-13 16:06:24,23114674.0,-0.969002,critical,severe
1,A00001767,C000984,購置機器設備,2011-07-29 00:59:54,2012-04-10 08:36:16,approved,4214153,3271448.25,營建工程,室內裝修,...,0.063283,-0.098891,0.012475,3.611103,32.043926,2014-08-25 16:19:12,24721708.0,-0.933689,high_risk,severe
2,A00000474,C000177,廠房擴充,2015-10-18 19:33:37,2016-02-05 23:45:04,approved,7983808,6556405.50,製造業,金屬加工,...,0.068842,-0.081491,0.012475,1.389789,16.307426,2018-02-22 22:41:04,20965120.0,4.104182,normal,normal
3,A00001885,C000343,廠房擴充,2017-10-09 18:18:43,2017-10-30 02:31:28,approved,18050866,10286025.00,營建工程,土木工程,...,0.052914,-0.084590,0.012475,0.951504,9.653728,2019-02-13 21:07:12,28994140.0,0.706520,high_risk,severe
4,A00001446,C000770,數位轉型,2018-03-03 20:07:18,2018-03-17 02:22:56,approved,16641708,11086960.00,製造業,塑膠製品,...,0.037981,-0.079716,0.012475,2.888257,31.203331,2019-11-14 17:12:32,27407920.0,1.350018,normal,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2510,A00002043,C000118,數位轉型,2014-01-03 04:25:07,2014-06-24 11:33:20,approved,3175286,1555414.75,服務業,專業諮詢,...,0.060019,-0.091203,0.012475,2.677720,31.400982,2017-01-04 20:45:52,24557100.0,4.946792,high_risk,normal
2511,A00000485,C000611,疫後紓困,2012-01-23 07:32:08,2012-10-13 16:34:08,approved,2284932,1388274.25,批發零售,零售,...,0.075867,-0.057222,0.012475,2.680346,31.532156,2014-07-25 00:51:12,24963926.0,-0.594900,normal,attention
2512,A00000857,C000045,購置機器設備,2016-05-13 18:00:06,2016-06-29 11:33:20,approved,2238610,1387708.25,批發零售,電子商務,...,0.045464,-0.094694,0.012475,3.360178,29.158670,2018-06-17 07:42:56,26096756.0,7.742785,warning,severe
2513,A00000758,C000264,營運週轉金,2014-04-30 06:15:30,2014-10-18 08:38:24,approved,14285570,9151632.00,批發零售,零售,...,0.063050,-0.043713,0.012475,1.290188,30.474125,2016-12-28 07:34:24,23032356.0,-0.947384,critical,normal
